**`Import necessary libraries`**

In [1]:
# import all the necessary libraries
import time
import cv2
from loguru import logger
from kafka import KafkaProducer
from Encoder_Decoder.encode_and_transmit import *

**`Load the video using opencv2`**


In [2]:
#load the video file
video_file='pexels-roman-odintsov-4927323-426x226-30fps.mp4'
cap = cv2.VideoCapture(video_file)

**`Setup a streaming server`**

In [3]:
bootstrap_servers = ['127.0.0.1:9093'] #hostname:port, this is the server where we will consume streaming data
topicName = 'video' #name of the kafka topic to subsribe to 

**`split the video into frames for streaming`**

*`Until there is no frame left in the video,`*  
*`send one frame every second to a Kafka server`*  
*`Apache kafka works with a publish and subscribe framework`*  
*`Producer publishes to a topic`*

In [4]:
#this callback is used to display error if occured while sending frames
def error_callback(exc):
      raise Exception('Error while sending data to kafka: {0}'.format(str(exc)))

#function to send video frames one by one every second
def Start_Streaming(topic_name):
    try:
        while(cap.isOpened()): # send frames until there are no frames left in the video
            ret, frame = cap.read()
            if ret == True: #if there is a frame to send
                producer = KafkaProducer(bootstrap_servers=bootstrap_servers) #connect to the streaming server
                #encode the frame to a base64 format
                #publish the frame to a Kafka topic named "video" 
                producer.send(topic_name, encode_and_transmit_numpy_array_in_bytes(frame).encode()) \
                    .add_errback(error_callback)
                time.sleep(1)
                producer.flush() #check if previous frame was sent succesfully
            else:
                break
        cap.release()
    except KeyboardInterrupt:
        logger.info("Streaming interrupted")


**`Stop or Pause the streaming by stopping the running cell`**

In [5]:
Start_Streaming('video')

2023-06-21 13:02:25.706 | INFO     | __main__:Start_Streaming:22 - Streaming interrupted
